# Alzheimer's disease dataset analysis and comparison

Obtaining and comparing gene sets thought to be compromised in Alzheimer's disease patients. 

In [1]:
# Importing packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import string


In [2]:
# Loading samples - Alzheimer's disease
ADsamples_Upenn_df = pd.read_table('AD_Upenn_prot.txt',index_col=0) 
ADsamples_Banner_df = pd.read_table('AD_Banner_prot.txt',index_col=0)
# AD_synap_plasticity = pd.read_csv('GO_REGULATION_OF_SYNAPTIC_PLASTICITY.csv', index_col = 0)

# AD_synap_plasticity
# ADsamples_Upenn_df
# ADsamples_Banner_df

In [3]:
# Importing genes from GO_REGULATION_OF_SYNAPTIC_PLASTICITY and creating list of genes

import csv

with open('GO_REGULATION_OF_SYNAPTIC_PLASTICITY.csv', 'r') as f:
    reader = csv.reader(f)
    gene_list_raw = list(reader)

gene_list_SP = []
for gene in gene_list_raw:
    gene_list_SP.append(gene[0]) 

In [15]:
# Taking intersection of all gene lists

# Lists from Alzheimer's data sets 
ADsamples_UPenn_gene_list = list(ADsamples_Upenn_df.dropna(axis='columns').columns.values)
ADsamples_Banner_gene_list = list(ADsamples_Banner_df.dropna(axis='columns').columns.values)

# taking intersection of lists - dropping NaN values
AD_genes_SP_intersection = list(set(gene_list_SP).intersection(ADsamples_UPenn_gene_list, ADsamples_Banner_gene_list))

print(len(AD_genes_SP_intersection))

44


In [19]:
# Intersections 

ADsamples_UPenn_gene_intersec_df = ADsamples_Upenn_df[AD_genes_SP_intersection]
ADsamples_Banner_gene_intersec_df = ADsamples_Banner_df[AD_genes_SP_intersection]

# number of NaN values
print(ADsamples_UPenn_gene_intersec_df.isnull().sum().sum())
print(ADsamples_Banner_gene_intersec_df.isnull().sum().sum())

# number of samples x genes with NaN values
# print(ADsamples_UPenn_gene_intersec_df.loc[:, ADsamples_UPenn_gene_intersec_df.isna().any()].shape)
# print(ADsamples_Banner_gene_intersec_df.loc[:, ADsamples_Banner_gene_intersec_df.isna().any()].shape)

# ADsamples_UPenn_gene_intersec_final_df = ADsamples_UPenn_gene_intersec_df.dropna(axis='columns')
# ADsamples_Banner_gene_intersec_final_df = ADsamples_Banner_gene_intersec_df.dropna(axis='columns')

0
0


In [18]:
ADsamples_Banner_gene_intersec_df.shape


(225, 44)